# UNET

<strong>Inspired by:</strong>
<ul>
    <li>https://www.kaggle.com/code/vikram12301/multiclass-semantic-segmentation-pytorch</li>
    <li>https://arxiv.org/pdf/1505.04597</li>
    <li>NVIDIA DLI Generative AI course </li>
</ul>


In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim import Adam
from torchvision.transforms import Compose, Resize, ToTensor, CenterCrop

import os
import re
from tqdm import tqdm

import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

In [2]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu"
device = torch.device(device)
device

device(type='cuda')

In [3]:
os.getcwd()

'/content'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data

In [5]:
class KeyboardDataset(Dataset):
    def __init__(self, img_dir, transform = None):
        self.transform = transform
        self.dir = img_dir
        all_images = os.listdir(img_dir)
        self.images = [img for img in all_images if re.search("Image", img)]

    def __len__(self):
        return len(self.images)

    def _get_mask_for_image(self,img):
        return img.replace("Image", "Segmentation")

    def __getitem__(self, index):
        img_path = self.images[index]
        img =  Image.open(os.path.join(self.dir, img_path)).convert("RGB")
        mask = Image.open(os.path.join(self.dir, self._get_mask_for_image(img_path)))
        if self.transform is not None:
            img, mask = self.transform(img), self.transform(mask)
        return img, mask

In [6]:
transforms = Compose(
    [
       CenterCrop(1080),
       Resize((256,256)),
       ToTensor()
    ]
)

In [7]:
img_dir = os.path.join("./drive/MyDrive", "only_keys_detection")

In [8]:
def get_images(image_dir, transform = None, batch_size = 1, shuffle = True, pin_memory=False):
    dataset = KeyboardDataset(img_dir=image_dir, transform=transform)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle, pin_memory=pin_memory)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle, pin_memory=pin_memory)
    return train_loader, test_loader

### Testing loaders

In [9]:
train_loader, test_loader = get_images(img_dir, transforms, batch_size=4)

In [10]:
for img,mask in train_loader:
    img1 = np.transpose(img[0,:,:,:],(1,2,0))
    mask1 = np.array(mask[0,:,:])
    img2 = np.transpose(img[1,:,:,:],(1,2,0))
    mask2 = np.array(mask[1,:,:])
    img3 = np.transpose(img[2,:,:,:],(1,2,0))
    mask3 = np.array(mask[2,:,:])
    fig , ax =  plt.subplots(3, 2, figsize=(18, 18))
    ax[0][0].imshow(img1)
    ax[0][1].imshow(np.squeeze(mask1))
    ax[1][0].imshow(img2)
    ax[1][1].imshow(np.squeeze(mask2))
    ax[2][0].imshow(img3)
    ax[2][1].imshow(np.squeeze(mask3))
    break

## Model

In [11]:
class RELUConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        layers = [
            nn.Conv2d(in_ch, out_ch,3,1,1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU()
        ]
        self.model = nn.Sequential(*layers)

    def forward(self,x):
        return self.model(x)

class DownBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        layers = [
            RELUConvBlock(in_ch, out_ch),
            RELUConvBlock(out_ch, out_ch)
        ]
        self.model = nn.Sequential(*layers)

    def forward(self,x):
        return self.model(x)

class UpBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        layers = [
            RELUConvBlock(in_ch, out_ch),
            RELUConvBlock(out_ch, out_ch)
        ]
        self.model = nn.Sequential(*layers)

    def forward(self,x):
        return self.model(x)

In [30]:
class UNet(nn.Module):
    def __init__(self, out_ch = 3, down_ch = [64,128,256,512]):
        super().__init__()
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.down0 = DownBlock(3, down_ch[0])
        self.down1 = DownBlock(down_ch[0], down_ch[1])
        self.down2 = DownBlock(down_ch[1], down_ch[2])
        self.down3 = DownBlock(down_ch[2], down_ch[3])

        self.bottleneck = DownBlock(down_ch[3], 2*down_ch[3])

        self.up3 = UpBlock(2*down_ch[-1], down_ch[-1])
        self.up2 = UpBlock(down_ch[-1], down_ch[-2])
        self.up1 = UpBlock(down_ch[-2], down_ch[-3])
        self.up0 = UpBlock(down_ch[-3], down_ch[-4])

        self.connect_b_up3 = nn.ConvTranspose2d(2*down_ch[-1], down_ch[-1],kernel_size=2,stride=2)
        self.connect_up3_up2 = nn.ConvTranspose2d(down_ch[-1], down_ch[-2],kernel_size=2,stride=2)
        self.connect_up2_up1 = nn.ConvTranspose2d(down_ch[-2], down_ch[-3],kernel_size=2,stride=2)
        self.connect_up1_up0 = nn.ConvTranspose2d(down_ch[-3], down_ch[-4],kernel_size=2,stride=2)

        self.final_conv = nn.Conv2d(down_ch[0], out_ch, kernel_size=1)

    def _crop_to_match(self, tensor, target):
        _, _, h, w = target.shape
        return tensor[:, :, :h, :w]

    def forward(self, x):
        skip_connections = []
        x = self.down0(x)
        skip_connections.append(x)
        x = self.pool(x)
        x = self.down1(x)
        skip_connections.append(x)
        x = self.pool(x)
        x = self.down2(x)
        skip_connections.append(x)
        x = self.pool(x)
        x = self.down3(x)
        skip_connections.append(x)
        x = self.pool(x)
        x = self.bottleneck(x)

        x = self.connect_b_up3(x)
        skip_connection = self._crop_to_match(skip_connections[3], x)
        x = torch.cat((skip_connection, x), dim=1)
        x = self.up3(x)
        x = self.connect_up3_up2(x)
        skip_connection = self._crop_to_match(skip_connections[2], x)
        x = torch.cat((skip_connection, x), dim=1)
        x = self.up2(x)
        x = self.connect_up2_up1(x)
        skip_connection = self._crop_to_match(skip_connections[1], x)
        x = torch.cat((skip_connection, x), dim=1)
        x = self.up1(x)
        x = self.connect_up1_up0(x)
        skip_connection = self._crop_to_match(skip_connections[0], x)
        x = torch.cat((skip_connection, x), dim=1)
        x = self.up0(x)

        x = self.final_conv(x)

        return x #nn.functional.sigmoid(x)


## Training

In [31]:
model = UNet().to(device)
print("Mun params:", sum(p.numel() for p in model.parameters()))

Mun params: 31043651


In [32]:
from torchsummary import summary

In [33]:
summary(model, (3,256,256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           1,792
       BatchNorm2d-2         [-1, 64, 256, 256]             128
              ReLU-3         [-1, 64, 256, 256]               0
     RELUConvBlock-4         [-1, 64, 256, 256]               0
            Conv2d-5         [-1, 64, 256, 256]          36,928
       BatchNorm2d-6         [-1, 64, 256, 256]             128
              ReLU-7         [-1, 64, 256, 256]               0
     RELUConvBlock-8         [-1, 64, 256, 256]               0
         DownBlock-9         [-1, 64, 256, 256]               0
        MaxPool2d-10         [-1, 64, 128, 128]               0
           Conv2d-11        [-1, 128, 128, 128]          73,856
      BatchNorm2d-12        [-1, 128, 128, 128]             256
             ReLU-13        [-1, 128, 128, 128]               0
    RELUConvBlock-14        [-1, 128, 1

In [34]:
LEARNING_RATE = 1e-4 #na potrzeby testy stały, ale może dodam go w postaci funcji epoki
num_epochs = 10

In [17]:
class DiceLoss(nn.Module): # to niepotzebne narazie
  def __init__(self) -> None:
      super().__init__()

  def forward(self, preds, target):
    return 1- (2 * (preds * target).sum() + 1e-8) / ((preds + target).sum() + 1e-8)

In [41]:
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)
scaler = torch.amp.GradScaler(device)

In [42]:
model.train()
for epoch in range(num_epochs):
    loop = tqdm(enumerate(train_loader), total = len(train_loader))
    for batch_idx, (data, targets) in loop:
        data = data.to(device)
        targets = targets * 3
        targets = targets.to(device)
        targets = targets.type(torch.long)
        #targets = F.one_hot(targets, num_classes = 3)
        targets = targets.squeeze(1)
        #targets = targets.permute(0,3,1,2)

        with torch.amp.autocast(device_type=str(device)):
            predictions = model(data)
            loss = loss_fn(predictions, targets)

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        loop.set_postfix(loss = loss.item())



100%|██████████| 83/83 [00:47<00:00,  1.74it/s, loss=0.249]


In [43]:
torch.save(model.state_dict(), "./drive/MyDrive/knr_model.pth")

## Metrics

In [44]:
def check_accuracy(loader, model):
  num_correct, num_pixels, dice_score = 0,0,0
  model.eval()

  with torch.no_grad():
    for x,y in loader:
      x = x.to(device)
      y = y.to(device)
      y = y.squeeze(1)
      softmax = nn.Softmax(dim = 1)
      preds = torch.argmax(softmax(model(x)), axis = 1)
      num_correct += (preds == y).sum()
      num_pixels += torch.numel(preds)
      dice_score += (2 * (preds * y).sum() + 1e-8) / ((preds + y).sum() + 1e-8)

  print(f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}")
  print(f"Dice score: {dice_score/len(loader)}")

In [45]:
check_accuracy(train_loader, model)

Got 12530670/21757952 with acc 57.59
Dice score: 1.0263820886611938


In [46]:
check_accuracy(test_loader, model)

Got 2952046/5505024 with acc 53.62
Dice score: 1.0261520147323608


In [47]:
for x,y in test_loader:
    x = x.to(device)
    fig , ax =  plt.subplots(3, 3, figsize=(18, 18))
    softmax = nn.Softmax(dim=1)
    preds = torch.argmax(softmax(model(x)),axis=1).to('cpu')
    img1 = np.transpose(np.array(x[0,:,:,:].to('cpu')),(1,2,0))
    preds1 = np.array(preds[0,:,:])
    mask1 = np.array(y[0,:,:])
    img2 = np.transpose(np.array(x[1,:,:,:].to('cpu')),(1,2,0))
    preds2 = np.array(preds[1,:,:])
    mask2 = np.array(y[1,:,:])
    img3 = np.transpose(np.array(x[2,:,:,:].to('cpu')),(1,2,0))
    preds3 = np.array(preds[2,:,:])
    mask3 = np.array(y[2,:,:])
    ax[0,0].set_title('Image')
    ax[0,1].set_title('Prediction')
    ax[0,2].set_title('Mask')
    ax[1,0].set_title('Image')
    ax[1,1].set_title('Prediction')
    ax[1,2].set_title('Mask')
    ax[2,0].set_title('Image')
    ax[2,1].set_title('Prediction')
    ax[2,2].set_title('Mask')
    ax[0][0].axis("off")
    ax[1][0].axis("off")
    ax[2][0].axis("off")
    ax[0][1].axis("off")
    ax[1][1].axis("off")
    ax[2][1].axis("off")
    ax[0][2].axis("off")
    ax[1][2].axis("off")
    ax[2][2].axis("off")
    ax[0][0].imshow(img1)
    ax[0][1].imshow(preds1)
    ax[0][2].imshow(np.squeeze(mask1))
    ax[1][0].imshow(img2)
    ax[1][1].imshow(preds2)
    ax[1][2].imshow(np.squeeze(mask2))
    ax[2][0].imshow(img3)
    ax[2][1].imshow(preds3)
    ax[2][2].imshow(np.squeeze(mask3))
    break